In [ ]:
import mikeio
import modelskill
import matplotlib.pyplot as plt 
import pandas as pd
from tqdm import tqdm
import numpy as np

In [ ]:
Flinten7 = pd.read_csv("raw_data/smhi-opendata_2_3_35137_20241118_065734.csv",index_col=0,sep=";", parse_dates=True)

NameError: name 'pd' is not defined